In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
%matplotlib inline

In [22]:
model_LGBM = lgb.LGBMRegressor()

In [26]:
df = pd.read_csv('data_train.csv')

In [27]:
df = df.drop(['Unnamed: 0', 'Тип топлива'], axis=1)
for col in df.columns:
    if df[col].dtypes == object:
        df[col] = df[col].astype('category')
        
        
new_names_col = ['brend', 'model', 'year', 'engine displacement',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']


dictionary = dict(zip(list(df.columns), new_names_col))
df = df.rename(columns=dictionary)


df.loc[df['year'] < 1994.0, ['year']] = None
df.loc[df['engine displacement'] > 3500.0, ['engine displacement']] = None
df.loc[df['engine displacement'] < 660.0, ['engine displacement']] = None
df.loc[df['hp'] < 40.0, ['hp']] = None
df.loc[df['hp'] > 500.0, ['hp']].loc[df['brend']=='volvo'] = None
df.loc[df['seats'] < 2, ['seats']] = None
df.loc[df['seats'] > 9, ['seats']] = None
df.loc[df['door'] < 2 , ['door']] = None
df.loc[df['door'] > 6  , ['door']] = None

df['seats'] = df['seats'].fillna(df['seats'].median())
df['door'] = df['door'].fillna(df['door'].median())
df['transmisson'] = df['transmisson'].replace(['ручной', 'авто'], [0, 1])

In [28]:
X_test_engine = df.loc[df['engine displacement'].isnull(), ['brend', 'model', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']]

X_train_engine = df.loc[df['engine displacement'].isnull() == False, ['brend', 'model', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price', 'engine displacement']]

y_train_engine = df.loc[df['engine displacement'].isnull() == False, ['engine displacement']]

In [32]:
model_LGBM.fit(X_train_engine.drop(['engine displacement'], axis=1), y_train_engine)
pred_engine = model_LGBM.predict(X_test_engine)

In [33]:
pred_engine = pd.Series(pred_engine,index=df.loc[df['engine displacement'].isnull(),'engine displacement'].index)
df['engine displacement'] = df['engine displacement'].fillna(pred_engine) // 1

In [34]:
df.head(10)

,brend,model,year,engine displacement,hp,transmisson,seats,door,mileage,price
0,audi,a8,2000.0,3328.000000,166.0,1,5.0,4.0,352640.00,101800
1,nissan,juke,2013.0,1520.941217,81.0,0,5.0,4.0,63460.00,250200
2,skoda,fabia,2014.0,1199.000000,55.0,0,5.0,5.0,15865.00,201700
3,audi,a3,2013.0,1968.000000,110.0,0,5.0,4.0,44606.30,491300
4,bmw,NaN,1999.0,2793.000000,142.0,0,5.0,4.0,187150.00,56100
5,ford,c-max,2015.0,998.000000,92.0,0,5.0,5.0,14487.50,292900
6,audi,a1,2012.0,1390.000000,136.0,1,4.0,2.0,23940.00,275100
7,skoda,octavia,2014.0,1993.607265,162.0,1,5.0,4.0,28500.00,440500
8,seat,alhambra,2014.0,1968.000000,103.0,0,5.0,4.0,27366.65,405800
9,bmw,NaN,2005.0,2171.000000,125.0,0,5.0,4.0,179075.00,120600
